In [ ]:
import pygame
from pygame.locals import *
from sys import exit

pygame.init()
clock = pygame.time.Clock()

In [ ]:
# Window

WIDTH: int = 640
HEIGHT: int = 480
SCREEN_SIZE: tuple = (WIDTH, HEIGHT)
TITLE: str = "Hello PyGame World!"
FIXED_FPS: int = 60

screen = pygame.display.set_mode(SCREEN_SIZE)
pygame.display.set_caption(TITLE)

In [ ]:
# Other imports
from numpy.linalg import norm
from random import randint

# Vector Constants
VECTOR_ZERO = (0, 0)
# VECTOR_RIGHT: tuple = (1, 0)
# VECTOR_LEFT: tuple = (-1, 0)
# VECTOR_UP: tuple = (0, 1)
# VECTOR_DOWN: tuple = (0, -1)

# Color Constants
COLOR_BLACK: tuple = (0, 0, 0)
COLOR_WHITE: tuple = (255, 255, 255)

In [ ]:
class InputEvent:

    def get_input_strength() -> list:
        is_pressed = pygame.key.get_pressed()
        keys: dict = {K_w: 0.0, K_a: 0.0, K_s: 0.0, K_d: 0.0}
        strength: list

        for key in keys:
            keys[key] = 1.0 if is_pressed[key] else 0.0

        strength = [keys[K_d] - keys[K_a], keys[K_s] - keys[K_w]]
        strength_norm = norm(strength)

        if strength_norm:
            strength /= strength_norm

        return strength


In [ ]:
class Entity:
    position: list
    color: tuple = (115, 10, 46)

    class SignalNotExists(Exception):
        pass

    class Signal:
        _observers: dict = {}
        owner = None

        class NotOwner(Exception):
            '''Lançado ao tentar operar o sinal para um objeto que não a pertence'''
            pass

        class AlreadyConnected(Exception):
            '''Lançada ao tentar conectar um sinal a um mesmo observador'''
            pass

        class NotConnected(Exception):
            '''Lançado ao tentar desconectar um sinal de um objeto que não é observador'''
            pass

        # TODO -> Allow args
        def connect(self, owner, observer, method) -> None:
            if owner != self.owner:
                raise Entity.Signal.NotOwner

            if self._observers.get(observer) != None:
                raise Entity.Signal.AlreadyConnected

            self._observers[observer] = method

        def disconnect(self, owner, observer) -> None:
            if owner != self.owner:
                raise Entity.Signal.NotOwner

            if self._observers.pop(observer) == None:
                raise Entity.Signal.NotConnected

        def emit(self, *args) -> None:
            for observer in self._observers.keys():
                self._observers[observer](*args)

        def __init__(self, owner) -> None:
            self.owner = owner

    def _draw(self):
        RECT_SIZE: int = 40
        HALF_SIZE: int = RECT_SIZE // 2  # Desloca a âncora para o centro
        RECT_TRANSFORM: tuple = (self.position[0] - HALF_SIZE,
                                 self.position[1] - HALF_SIZE, RECT_SIZE, RECT_SIZE)

        return pygame.draw.rect(screen, self.color, RECT_TRANSFORM)

    def connect(self, signal, observer, method) -> None:
        try:
            signal.connect(self, observer, method)
        except Entity.Signal.NotOwner:
            raise Entity.SignalNotExists

    def disconnect(self, signal, observer, method) -> None:
        try:
            signal.disconnect(self, observer)
        except Entity.Signal.NotOwner:
            raise Entity.SignalNotExists

    def get_x(self) -> int:
        return self.position[0]

    def get_y(self) -> int:
        return self.position[1]

    def __init__(self, coords: tuple = VECTOR_ZERO) -> None:
        self.position = list(coords)


In [ ]:
class KinematicBody(Entity):
    speed: float = 1.0
    velocity: list = [0.0, 0.0]

    def move(self) -> None:

        for i in range(2):
            self.position[i] += self.velocity[i] * self.speed

            if self.position[i] < 0.0:
                self.position[i] = 0.0
            elif self.position[i] > SCREEN_SIZE[i]:
                self.position[i] = SCREEN_SIZE[i]

    def _input(self) -> None:
        self.velocity = InputEvent.get_input_strength()

    # def _input_event(self, event: InputEvent) -> None:
    #     pass

    def __init__(self, coords: tuple, color=(46, 10, 115)) -> None:
        super().__init__(coords)
        self.color = color


In [ ]:
class Label(Entity):
    font = pygame.font.SysFont('roboto', 40, False, False)
    text: str = ''

    def set_text(self, value: str) -> None:
        self.text = value

    def _draw(self):
        return self.font.render(self.text, True, self.color)
    
    def __init__(self, coords: tuple = VECTOR_ZERO, color = COLOR_WHITE) -> None:
        super().__init__(coords=coords)
        self.color = color


In [ ]:
class Player(KinematicBody):
    points_changed: Entity.Signal
    
    def set_points(self, value) -> None:
        self._points = value
        self.points_changed.emit(f'Points: {value}')
    
    def get_points(self) -> None:
        return self._points
    
    def __init__(self, coords: tuple, color=(15, 92, 105)) -> None:
        super().__init__(coords, color=color)
        self._points = 0
        self.points_changed = Entity.Signal(self)
    
    points = property(get_points, set_points)

In [ ]:
# Entities
player: Player = Player((WIDTH // 2, HEIGHT // 2))
entity: Entity = Entity((randint(0, WIDTH), randint(0, HEIGHT)))
label: Label = Label((450, 40))

# Sound Streams
SOUNDS_DIR: str = 'assets/sounds'
MUSIC_DIR: str = f'{SOUNDS_DIR}/music'
SFX_DIR: str = f'{SOUNDS_DIR}/sfxs'
bgm = pygame.mixer.music.load(f'{MUSIC_DIR}/IPAGHOST - Not All Friends Are Ice Cream Cones.ogg')
pygame.mixer.music.set_volume(0.1)
pygame.mixer.music.play(-1)
sfx = pygame.mixer.Sound(f'{SFX_DIR}/checkpoint.wav')

label.text = 'Points: 0'
player.connect(player.points_changed, label, label.set_text)

while True:
    clock.tick(FIXED_FPS)
    screen.fill(COLOR_BLACK)

    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            exit()

#         if event.type == KEYDOWN:
#             keys: list = [K_w, K_a, K_s, K_d]
#
#             for key in keys:
#                 if event.key == key:
#                     player._input_event(
#                         InputEvent.InputTypes.JUST_PRESSED, key)

    player._input()
    player.move()
    body_aabb = player._draw()
    entity_aabb = entity._draw()
    screen.blit(label._draw(), label.position)
    
    if body_aabb.colliderect(entity_aabb):
        entity.position = (randint(0, WIDTH), randint(0, HEIGHT))
        player.points += 1
        sfx.play()
    
    pygame.display.update()
